In [ ]:
#!pip install mysql-connector-python

**Выполнение тестового задания**

Вам нужно написать код на python, который сделает дедубликацию торговых точек. Вы можете использовать свой алгоритм и/или сторонние(внешние) сервисы.
Пожалуйста, предоставьте ссылку на репозиторий Github(Bitbucket), где будет находиться Ваш код и конечный результат(SQL таблицы с дедублицированными данными).

SQL таблицы - https://drive.google.com/file/d/0B6nelhokQOrRZkMzWHgzSWFBaUE/view

    Вам нужно будет заполнить таблицу “outlets_clean” и обновить внешний ключ в таблице “outlets”(колонка outlet_clean_id). Для точек в таблице “outlets”, которые Вы не можете дедублицировать, проставьте в колонке outlet_clean_id значение NULL.

In [1]:
import mysql.connector
from mysql.connector import Error
import pymorphy2
import pandas as pd
import seaborn as sns

morph = pymorphy2.MorphAnalyzer()

Функция для подключения к локальному MySQL

In [2]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
connection = create_connection("localhost", "root", "123555", "sm_app")

Connection to MySQL DB successful


Функция для чтения отношений из бд

In [4]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [5]:
select_outlets = "SELECT * FROM outlets"
string_df = execute_read_query(connection, select_outlets)
df_outline = pd.DataFrame(string_df)
df_outline[:5]

,0,1,2,3,4
0,1,Ростов-на-Дону,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",None
1,2,Ростов-на-Дону,ИП Малышева,-,None
2,3,Ростов-на-Дону,Абасян А.П. ИП,он же,None
3,4,Ростов-на-Дону,"Азатян Н,С. ИП ул. Ленина",он же,None
4,5,Ростов-на-Дону,Айдинов В.Я.,он же,None


In [6]:
df_outline.columns = ['id', 'city', 'name', 'address', 'outlets_clean']

In [7]:
df_outline[:5]

,id,city,name,address,outlets_clean
0,1,Ростов-на-Дону,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",None
1,2,Ростов-на-Дону,ИП Малышева,-,None
2,3,Ростов-на-Дону,Абасян А.П. ИП,он же,None
3,4,Ростов-на-Дону,"Азатян Н,С. ИП ул. Ленина",он же,None
4,5,Ростов-на-Дону,Айдинов В.Я.,он же,None


In [8]:
df_outline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20208 entries, 0 to 20207
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             20208 non-null  int64 
 1   city           20208 non-null  object
 2   name           20208 non-null  object
 3   address        20208 non-null  object
 4   outlets_clean  0 non-null      object
dtypes: int64(1), object(4)
memory usage: 789.5+ KB


Проверка на явные дубликаты

In [9]:
df_outline[['name', 'city', 'address']].duplicated().sum()

3269

Как минимум у нас 3269 явных дубликатов

In [10]:
df_outline.loc[df_outline[['name', 'city', 'address']].duplicated()].sort_values(by='city' ,ascending=False)

,id,city,name,address,outlets_clean
66,67,Ростов-на-Дону,Виноградова,"г.Батайск,Авиагородок",None
18536,18537,Ростов-на-Дону,Каскад ООО,"347413, Ростовская обл, Дубовский р-н, Андреев...",None
18525,18526,Ростов-на-Дону,"Карманов С.А. ИП Большой пр-кт,48","347930, Ростовская обл, Таганрог г, Большой пр...",None
18527,18528,Ростов-на-Дону,Карпов А.С.ИП,-,None
18528,18529,Ростов-на-Дону,Карпов Г.А. ИП,"347042, Ростовская обл, Белая Калитва г, Калин...",None
...,...,...,...,...,...
10200,10201,Ростов-на-Дону,Саркисян С А,-,None
10202,10203,Ростов-на-Дону,Саркисян С.А.,г.Ростов-на-Дону Забайкальский 4,None
10203,10204,Ростов-на-Дону,Саркисян С.А.,г.Ростов-на-Дону Забайкальский 4,None
10208,10209,Ростов-на-Дону,Свиридова Л А ИП,-,None


In [11]:
df_outline['city'].unique()

array(['Ростов-на-Дону'], dtype=object)

Выборка производится по одному городу, на него можно не смотреть.

In [12]:
display(len(df_outline['address'].unique()))
len(df_outline['name'].unique())


7414

11075

Количество уникальных значений в адресах и именах

In [13]:
df_outline['address'] = df_outline['address'].str.lower()
df_outline['name'] = df_outline['name'].str.lower()
display(len(df_outline['address'].unique()))
len(df_outline['name'].unique())

7403

10936

Нашли небольшое кол-во неявных дубликатов.

Приведем слова к начальной форме и попробуем найти еще больше неявных дубликатов.

In [14]:
def lem_function(text):
    normal_word = ''
    for word in text.split(' '):
        p = morph.parse(word)[0]  # делаем разбор
        normal_word += p.normal_form + ' '
    return normal_word.strip()

In [15]:
#'это долго'
df_outline['address'] = df_outline['address'].apply(lem_function)
df_outline['name'] = df_outline['name'].apply(lem_function)
display(len(df_outline['address'].unique()))
len(df_outline['name'].unique())

7403

10932

Нашли еще четыре дубликата

In [16]:
df_outline

,id,city,name,address,outlets_clean
0,1,Ростов-на-Дону,ип гаврил с.а.,"п.глубокий, ул.артём 44",None
1,2,Ростов-на-Дону,ип малышев,-,None
2,3,Ростов-на-Дону,абасян а.п. ип,он же,None
3,4,Ростов-на-Дону,"азатян н,с. ип ул. ленин",он же,None
4,5,Ростов-на-Дону,айдин в.я.,он же,None
...,...,...,...,...,...
20203,20204,Ростов-на-Дону,яров а.т.ип,"346535, ростовский обл, шахта г, кошевой ул, д...",None
20204,20205,Ростов-на-Дону,"яров в.т. ип м-н ""весна""",-,None
20205,20206,Ростов-на-Дону,ярута с.м. ип,"347332, ростовский область, волгодонский р-н, ...",None
20206,20207,Ростов-на-Дону,яхон б.а. ип,"346380, ростовский обл, красносулинский р-н, л...",None


Запишем айди уникальных значений в outlets_clean

In [17]:
df_outline = df_outline.sort_values(by=['name', 'address'])

In [18]:
df_outline.loc[100:150][['name', 'address']].head(20)

,name,address
100,гридин н.а. ип,он же
12467,"гридин н.а. ип садовая, 48",-
12468,"гридин н.а. ип садовая, 48","347850, ростовский обл, каменский р-н, глубоки..."
18116,"гридин н.а. ип садовая, 48","347850, ростовский обл, каменский р-н, глубоки..."
101,"гридин н.а. ип садовая, 48",он же
12470,"гриневич в.м. ип ул.фрунзе/маяковского, 68/58","346400, ростовский обл, новочеркасск г, первом..."
12471,гриненко т.в. ип,"359010, калмыкия респ, яшалтинский р-н, яшалта..."
18118,гриненко т.в. ип,"359010, калмыкия респ, яшалтинский р-н, яшалта..."
1992,гринь,-
12469,гринёв г.л.ип,"346061, ростовский обл, тарасовский р-н, марты..."


В таблице присутствует такое значение адреса как "он же". По хорошему за такое руки бы отрывать. Будем исходить из предположения о том, что предыдущий адрес соответствует адресу в строке "он же".

In [19]:
filter = df_outline['address'] == 'он же'
df_outline[filter]   

,id,city,name,address,outlets_clean
2,3,Ростов-на-Дону,абасян а.п. ип,он же,None
3,4,Ростов-на-Дону,"азатян н,с. ип ул. ленин",он же,None
4,5,Ростов-на-Дону,айдин в.я.,он же,None
6,7,Ростов-на-Дону,алексеев л п ип,он же,None
8,9,Ростов-на-Дону,алпатов в в ип,он же,None
...,...,...,...,...,...
1025,1026,Ростов-на-Дону,яковлев а.в. ип,он же,None
1026,1027,Ростов-на-Дону,якубенко м.с. ип,он же,None
1027,1028,Ростов-на-Дону,ярмоновый н.г.ип,он же,None
1029,1030,Ростов-на-Дону,"яров в.т. ип м-н ""весна""",он же,None


In [20]:
df_outline = df_outline.sort_values(by='id')

В таком случае считаем что адрес "он же" после "-" - не индифицирован. Потому что не ясно к какому адресу относится.

In [21]:
df = df_outline.copy()

In [22]:
for i in range(len(df.index)):
    if df.loc[i]['address'] == 'он же':
        df.loc[i, 'address'] = df.loc[i-1, 'address']        

Отсортируем и проиндексируем уникальные значения.

In [23]:
df = df.sort_values(by=['name'])

Для глубокого  нахождения дубликатов воспользуемся сторонней библиотекой https://github.com/Lyonk71/pandas-dedupe. Здесб используется обучаемая с учителем нейросеть для поиска дубликатов. В задании указано что поиск дубликатов должен происходить по названию торговых точек. 

In [ ]:
#!pip install git+https://github.com/Lyonk71/pandas-dedupe.git

In [24]:
import pandas_dedupe 

In [25]:
df_final = pandas_dedupe.dedupe_dataframe(df,['name'])

Importing data ...


C:\Users\sveto\AppData\Local\Temp\ipykernel_2728\3718420957.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_final = pandas_dedupe.dedupe_dataframe(df,['name'])


Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 11518


In [26]:
df_final.sort_values(by='id')

,id,city,name,address,outlets_clean,cluster id,confidence
0,1,rostov-na-donu,ip gavril s.a.,"p.glubokii, ul.artiom 44",None,2035,0.797475
9,10,rostov-na-donu,alpatov v v ip,ul gvardeiskii 34-a,None,5033,1.000000
99,100,rostov-na-donu,grigorian sh.g.,g. rostov-na-donu ul. ts. kunnikova 8b,None,321,0.727990
999,1000,rostov-na-donu,kholodenko i.v.ip m-n eva,-,None,366,0.641944
9999,10000,rostov-na-donu,"pavlov n.i. ip, gukovo",-,None,2354,0.705073
...,...,...,...,...,...,...,...
9994,9995,rostov-na-donu,"pavlenko a.v. ip (kagal'nits.r-n) 4, m.batai",-,None,1190,0.788261
9995,9996,rostov-na-donu,pavlenko a.v. ip (kagal'nits.r-n) 5,-,None,1190,0.792105
9996,9997,rostov-na-donu,pavlov n i ip,-,None,2355,0.899337
9997,9998,rostov-na-donu,pavlov n.i. ip,-,None,2354,0.748016


In [27]:
df_final = df_final.astype({'id': int})

Теперь присоединим эту таблицу по айдишникам к исходной таблице.

In [28]:
df_outline = df_outline.merge(df_final[['id', 'cluster id']], on='id')

In [29]:
df_outline = df_outline.drop(columns='outlets_clean', axis=1)

In [30]:
df_outline.rename(columns={'city' : 'Город дистрибьютора'
                  , 'name' : 'Торг_точка_грязная'
                  , 'address' : 'Торг_точка_грязная_адрес'
                  , 'cluster id' : 'outlet_clean_id'}, inplace=True)

In [31]:
df_outline

,id,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
0,1,Ростов-на-Дону,ип гаврил с.а.,"п.глубокий, ул.артём 44",2035
1,2,Ростов-на-Дону,ип малышев,-,2036
2,3,Ростов-на-Дону,абасян а.п. ип,он же,2038
3,4,Ростов-на-Дону,"азатян н,с. ип ул. ленин",он же,315
4,5,Ростов-на-Дону,айдин в.я.,он же,5030
...,...,...,...,...,...
20203,20204,Ростов-на-Дону,яров а.т.ип,"346535, ростовский обл, шахта г, кошевой ул, д...",2644
20204,20205,Ростов-на-Дону,"яров в.т. ип м-н ""весна""",-,2645
20205,20206,Ростов-на-Дону,ярута с.м. ип,"347332, ростовский область, волгодонский р-н, ...",2025
20206,20207,Ростов-на-Дону,яхон б.а. ип,"346380, ростовский обл, красносулинский р-н, л...",2646


In [33]:
for i in range(len(df_outline.index)):
    if df_outline.loc[i]['Торг_точка_грязная_адрес'] == 'он же':
        df_outline.loc[i, 'Торг_точка_грязная_адрес'] = df_outline.loc[i-1, 'Торг_точка_грязная_адрес']    

In [34]:
outlets_clean = df_final.sort_values(by='id').pivot_table(index='cluster id', values='id', aggfunc='first').reset_index()

In [35]:
outlets_clean = outlets_clean.merge(df_outline[['Торг_точка_грязная_адрес', 'id']], on='id' )

In [36]:
outlets_clean.drop(columns='id', axis=1, inplace=True)

In [37]:
outlets_clean = outlets_clean.rename(columns={'cluster id' : 'id', 'Торг_точка_грязная_адрес' : 'Торг_точка_чистый_адрес'})

In [38]:
df_outline.sort_values(by='outlet_clean_id')

,id,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
4251,4252,Ростов-на-Дону,семя дон зао /2,-,0
4252,4253,Ростов-на-Дону,семя дон зао,-,0
15722,15723,Ростов-на-Дону,семя дон зао нпф ул. красный,"344002, ростовский обл, ростов-на-дону г, будё...",1
15721,15722,Ростов-на-Дону,семя дон зао нпф ул. красный,-,1
4273,4274,Ростов-на-Дону,серикова н.м. ип\к.лучко,г.краснодар ул.клара лучко 6,2
...,...,...,...,...,...
20178,20179,Ростов-на-Дону,яковенко с.в.ип,"346109, ростовский обл, миллеровский р-н, юго-...",11513
20183,20184,Ростов-на-Дону,яковлев ю.а.ип,"346735, ростовский обл, аксайский р-н, рассвет...",11514
20197,20198,Ростов-на-Дону,яровой д.а. ип м-н уголёк,"346535, ростовский обл, шахта г, победный ул, ...",11515
20202,20203,Ростов-на-Дону,"яров а.н. ип ул.потаржинского,3","346500, ростовский обл, шахта г, потаржинский ...",11516


In [40]:
display(outlets_clean)
df_outline.sort_values(by='id')

,id,Торг_точка_чистый_адрес
0,0,-
1,1,-
2,2,-
3,3,350072 г.краснодар ул. ростовский шоссе 25/2
4,4,350072 г краснодар ул ростовский шоссе 25/2
...,...,...
11513,11513,"346109, ростовский обл, миллеровский р-н, юго-..."
11514,11514,"346735, ростовский обл, аксайский р-н, рассвет..."
11515,11515,"346535, ростовский обл, шахта г, победный ул, ..."
11516,11516,"346500, ростовский обл, шахта г, потаржинский ..."


,id,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
0,1,Ростов-на-Дону,ип гаврил с.а.,"п.глубокий, ул.артём 44",2035
1,2,Ростов-на-Дону,ип малышев,-,2036
2,3,Ростов-на-Дону,абасян а.п. ип,-,2038
3,4,Ростов-на-Дону,"азатян н,с. ип ул. ленин",-,315
4,5,Ростов-на-Дону,айдин в.я.,-,5030
...,...,...,...,...,...
20203,20204,Ростов-на-Дону,яров а.т.ип,"346535, ростовский обл, шахта г, кошевой ул, д...",2644
20204,20205,Ростов-на-Дону,"яров в.т. ип м-н ""весна""",-,2645
20205,20206,Ростов-на-Дону,ярута с.м. ип,"347332, ростовский область, волгодонский р-н, ...",2025
20206,20207,Ростов-на-Дону,яхон б.а. ип,"346380, ростовский обл, красносулинский р-н, л...",2646


Обе таблички готовы. Осталось, согласно задания, записать их в базу данных.

In [42]:
insert_reviewers_query = """
INSERT INTO outlets_clean
(id,Торг_точка_чистый_адрес )
VALUES ( %s, %s )
"""
reviewers_records = tuple(outlets_clean.itertuples(index=False, name=None))

with connection.cursor() as cursor:
    cursor.executemany(insert_reviewers_query,
                       reviewers_records)
    connection.commit()

In [55]:
outlets_id = []
df_outlines = df_outline.sort_values(by='id')
for i in df_outline['outlet_clean_id']:
    temp_list =[i]
    outlets_id.append(temp_list)

In [61]:
df_outline.to_csv('111.csv')

In [64]:
insert_reviewers_query = """
INSERT INTO outlets
(outlet_clean_id )
VALUES ( %s)
"""
reviewers_records = outlets_id

with connection.cursor() as cursor:
    cursor.executemany(insert_reviewers_query,
                       reviewers_records)
    connection.commit()

Вот как-то так.